# Resnet50, Mobilenet ve Densenet

In [1]:
# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [1]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_resnet
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as preprocess_mobilenet
from keras.applications.xception import Xception, preprocess_input as preprocess_xception
from sklearn.model_selection import train_test_split
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')

# Veri yolu
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Stem Sağlıklı Grafik"
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/Stem Okralı Grafik"

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Dosya adlarını saklamak için listeleri oluştur
zar_filenames = []
nor_filenames = []

# Zararlı sınıf için verileri yükleme
for dosya_adi in os.listdir(okra_klasoru):
    dosya_yolu = os.path.join(okra_klasoru, dosya_adi)
    img = load_img(dosya_yolu, target_size=(256, 256))
    img = img_to_array(img)
    img = preprocess_resnet(img)  # ResNet50 için ön işleme
    X.append(img)
    y.append(0)  # Zararlı sınıf
    zar_filenames.append(dosya_adi)

# Normal sınıf için verileri yükleme
for dosya_adi in os.listdir(normal_klasoru):
    dosya_yolu = os.path.join(normal_klasoru, dosya_adi)
    img = load_img(dosya_yolu, target_size=(256, 256))
    img = img_to_array(img)
    img = preprocess_resnet(img)  # ResNet50 için ön işleme
    X.append(img)
    y.append(1)  # Normal sınıf
    nor_filenames.append(dosya_adi)

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Model listesi
models = [
    ("ResNet50", ResNet50, preprocess_resnet),
    ("MobileNetV2", MobileNetV2, preprocess_mobilenet),
    ("Xception", Xception, preprocess_xception)
]

# Model döngüsü
for model_name, model_type, preprocess_func in models:
    # Modeli yükleme
    base_model = model_type(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Modelin üst kısmını oluşturma
    from keras.layers import Flatten, Dense
    from keras.models import Model

    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    # Yeni model için ayrı bir isim kullan
    new_model = Model(inputs=base_model.input, outputs=output)

    # Modeli derleme
    new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Eğitim ve test setlerini oluştur
    X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modeli eğitme
    new_model.fit(preprocess_func(X_train_model), y_train_model, epochs=20, batch_size=32, validation_split=0.1)

    # Modeli test verisiyle değerlendirme
    loss, accuracy = new_model.evaluate(preprocess_func(X_test_model), y_test_model)
    print(f'{model_name} Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')

    # Tahminleri saklamak için boş bir dizi oluşturma
    tahminler = []

    # Test görüntülerinin tahminlerini alın
    for i in range(len(X_test_model)):
        tahmin = new_model.predict(np.expand_dims(preprocess_func(X_test_model[i]), axis=0))
        tahminler.append((zar_filenames[i], nor_filenames[i], tahmin[0][0], y_test_model[i]))

    # Tahminleri doğruluk oranına göre sırala
    tahminler.sort(key=lambda x: abs(x[2] - x[3]))

    # Doğruluk oranı ve tahminleri yazdırma
    print(f"{model_name} Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%")
    print(f"{model_name} Modeli Tahminler (Dosya Adı Zararlı, Dosya Adı Normal, Tahmin, Gerçek Etiket):")
    for tahmin in tahminler:
        print(tahmin)


Mounted at /content/gdrive
94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/20
23/23 [==============================] - 56s 613ms/step - loss: 6.1581 - accuracy: 0.4875 - val_loss: 71.7653 - val_accuracy: 0.4125
Epoch 2/20
23/23 [==============================] - 8s 364ms/step - loss: 0.6932 - accuracy: 0.4958 - val_loss: 14.2899 - val_accuracy: 0.4125
Epoch 3/20
23/23 [==============================] - 8s 369ms/step - loss: 0.6933 - accuracy: 0.4917 - val_loss: 0.6925 - val_accuracy: 0.5750
Epoch 4/20
23/23 [==============================] - 8s 369ms/step - loss: 0.6932 - accuracy: 0.4958 - val_loss: 0.6931 - val_accuracy: 0.5875
Epoch 5/20
23/23 [==============================] - 9s 374ms/step - loss: 0.6932 - accuracy: 0.4958 - val_loss: 0.6931 - val_accuracy: 0.5875
Epoch 6/20
23/23 [==============================] - 9s 377ms/step - loss: 0.6932 - accuracy: 0.4819 - val_loss: 0.6932 - val_accuracy: 0.4125
Epoch 7/20
23/23 [==============================] - 9

('Column 1_164210_20240406_184542_a.png', 'Column 1_164023_20240406_164732_a.png', 0.5022384, 1)
('Column 1_164211_20240406_184620_a.png', 'Column 1_164025_20240406_164809_a.png', 0.5022384, 1)
('Column 1_164216_20240406_184934_a.png', 'Column 1_164029_20240406_165018_a.png', 0.5022384, 1)
('Column 1_164218_20240406_185127_a.png', 'Column 1_164031_20240406_165113_a.png', 0.5022384, 1)
('Column 1_164221_20240406_185304_a.png', 'Column 1_164036_20240406_165548_a.png', 0.5022384, 1)
('Column 1_164223_20240406_185427_a.png', 'Column 1_164038_20240406_165627_a.png', 0.5022384, 1)
('Column 1_164222_20240406_185404_a.png', 'Column 1_164040_20240406_165747_a.png', 0.5022384, 1)
('Column 1_164228_20240406_185813_a.png', 'Column 1_164048_20240406_170158_a.png', 0.5022384, 1)
('Column 1_164229_20240406_185903_a.png', 'Column 1_164053_20240406_170431_a.png', 0.5022384, 1)
('Column 1_164234_20240406_190227_a.png', 'Column 1_164055_20240406_170541_a.png', 0.5022384, 1)
('Column 1_164232_20240406_190

# EN YÜKSEK DOĞRULUK ORANINA SAHİP ALGORİTMA İLE SONUCUNU BASIYOR, HEPSİNİ SIRAYLA GÖSTERMİYOR

In [ ]:
# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# BU KOD SADECE EN YÜKSEK ORANI VEREN ALGORİTMAYI VE DOĞRULUK ORANINI BASTIRIYOR

import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_resnet
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as preprocess_mobilenet
from keras.applications.xception import Xception, preprocess_input as preprocess_xception
from sklearn.model_selection import train_test_split

# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')

# Veri yolu
zar_klasor = '/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/30Data_Graphs_40_228/Okrali_Graphs'
nor_klasor = '/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Grafikler/30Data_Graphs_40_228/Normal_Graphs'

# Veri ve etiketlerin depolanacağı boş listeleri oluştur
X = []
y = []

# Dosya adlarını saklamak için listeleri oluştur
zar_filenames = []
nor_filenames = []

# Zararlı sınıf için verileri yükleme
for dosya_adi in os.listdir(zar_klasor):
	dosya_yolu = os.path.join(zar_klasor, dosya_adi)
	img = load_img(dosya_yolu, target_size=(224, 224))
	img = img_to_array(img)
	img = preprocess_resnet(img)  # ResNet50 için ön işleme
	X.append(img)
	y.append(0)  # Zararlı sınıf
	zar_filenames.append(dosya_adi)

# Normal sınıf için verileri yükleme
for dosya_adi in os.listdir(nor_klasor):
	dosya_yolu = os.path.join(nor_klasor, dosya_adi)
	img = load_img(dosya_yolu, target_size=(224, 224))
	img = img_to_array(img)
	img = preprocess_resnet(img)  # ResNet50 için ön işleme
	X.append(img)
	y.append(1)  # Normal sınıf
	nor_filenames.append(dosya_adi)

# Veriyi Numpy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Eğitim ve test verisi olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelleri yükleme
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
xception_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Modeli eğitilemez hale getirme
for model in [resnet_model, mobilenet_model, xception_model]:
	model.trainable = False

# Model listesi
models = [("ResNet50", resnet_model, preprocess_resnet),
      	("MobileNetV2", mobilenet_model, preprocess_mobilenet),
      	("Xception", xception_model, preprocess_xception)]

# Model döngüsü
for model_name, model, preprocess_func in models:
	# Tam bağlı katmanları oluşturma
	from keras.layers import Flatten, Dense
	from keras.models import Model

x = Flatten()(model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=model.input, outputs=output)

# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(preprocess_func(X_train), y_train, epochs=20, batch_size=32, validation_split=0.1)

# Modeli test verisiyle değerlendirme
loss, accuracy = model.evaluate(preprocess_func(X_test), y_test)
print(f'{model_name} Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')

# Tahminleri saklamak için boş bir dizi oluşturma
tahminler = []

# Test görüntülerinin tahminlerini alın
for i in range(len(X_test)):
    tahmin = model.predict(np.expand_dims(preprocess_func(X_test[i]), axis=0))
    tahminler.append((zar_filenames[i], nor_filenames[i], tahmin[0][0], y_test[i]))

# Tahminler listesini doğruluk oranına göre sıralama
tahminler.sort(key=lambda x: abs(x[2] - x[3]))

# Doğruluk oranı ve tahminleri yazdırma
print(f'{model_name} Modeli Test Doğruluk Oranı: {accuracy * 100:.2f}%')
print(f"{model_name} Modeli Tahminler (Dosya Adı Zararlı, Dosya Adı Normal, Tahmin, Gerçek Etiket):")
for tahmin in tahminler:
   	print(tahmin)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch 1/20
2/2 [==============================] - 17s 7s/step - loss: 10.3905 - accuracy: 0.4884 - val_loss: 7.7911 - val_accuracy: 0.4000
Epoch 2/20
2/2 [==============================] - 8s 3s/step - loss: 3.5276 - accuracy: 0.6047 - val_loss: 6.7963 - val_accuracy: 0.6000
Epoch 3/20
2/2 [==============================] - 10s 3s/step - loss: 3.0373 - accuracy: 0.7442 - val_loss: 7.9890 - val_accuracy: 0.6000
Epoch 4/20
2/2 [==============================] - 10s 4s/step - loss: 2.8453 - accuracy: 0.8140 - val_loss: 8.6747 - val_accuracy: 0.6000
Epoch 5/20
2/2 [==============================] - 8s 3s/step - loss: 1.8192 - accuracy: 0.8605 - val_loss: 7.5580 - val_accuracy: 0.6000
Epoch 6/20
2/2 [==============================] - 11s 3s/step - loss: 0.8549 - accuracy: 0.8372 - val_loss: 6.8458 - val_accuracy: 0.6000
Epoch 7/20
2/2 [==========================